## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-30-01-12-48 +0000,nyt,Geert Wilders’s Party for Freedom Loses Seats ...,https://www.nytimes.com/2025/10/29/world/europ...
1,2025-10-30-01-07-43 +0000,nyt,Jamaica and Haiti Tally Hurricane Victims as M...,https://www.nytimes.com/2025/10/29/world/ameri...
2,2025-10-30-01-06-43 +0000,nypost,Hundreds of dedicated birders flock to Long Is...,https://nypost.com/2025/10/29/us-news/hundreds...
3,2025-10-30-01-05-48 +0000,nyt,Live Updates: Toll Rises as Hurricane Melissa ...,https://www.nytimes.com/live/2025/10/29/weathe...
4,2025-10-30-01-01-02 +0000,wapo,Trump says South Korea will be able to build n...,https://www.washingtonpost.com/world/2025/10/2...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2343/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
35,trump,45
418,new,28
38,will,16
11,hurricane,16
13,melissa,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
404,2025-10-29-02-10-44 +0000,nypost,North Korea boasts missile test-firings as Tru...,https://nypost.com/2025/10/28/world-news/north...,98
163,2025-10-29-20-09-08 +0000,wapo,"Trump rebukes Putin for testing new missiles, ...",https://www.washingtonpost.com/world/2025/10/2...,98
99,2025-10-29-21-43-10 +0000,nypost,Trump confident cease-fire will hold after Isr...,https://nypost.com/2025/10/29/world-news/trump...,96
262,2025-10-29-15-17-29 +0000,nyt,"Feted as a Peacemaker in South Korea, Trump Vo...",https://www.nytimes.com/2025/10/29/world/asia/...,95
4,2025-10-30-01-01-02 +0000,wapo,Trump says South Korea will be able to build n...,https://www.washingtonpost.com/world/2025/10/2...,94


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
404,98,2025-10-29-02-10-44 +0000,nypost,North Korea boasts missile test-firings as Tru...,https://nypost.com/2025/10/28/world-news/north...
97,75,2025-10-29-21-43-47 +0000,nypost,UK paid $41M to shoot new ‘Knives Out’ Netflix...,https://nypost.com/2025/10/29/media/uk-paid-41...
211,66,2025-10-29-18-05-17 +0000,wapo,Report: Government shutdown will cost the econ...,https://www.washingtonpost.com/business/2025/1...
137,61,2025-10-29-20-56-14 +0000,wapo,Hurricane Melissa hits Cuba; has killed at lea...,https://www.washingtonpost.com/weather/2025/10...
73,51,2025-10-29-22-41-05 +0000,nypost,Zohran Mamdani’s DSA allies make last ditch pu...,https://nypost.com/2025/10/29/us-news/zohran-m...
86,50,2025-10-29-22-14-02 +0000,nypost,Teen ‘stalker’ charged in fatal hit-and-run of...,https://nypost.com/2025/10/29/us-news/teen-cha...
206,49,2025-10-29-18-09-32 +0000,nypost,Fed cuts interest rates by quarter point for s...,https://nypost.com/2025/10/29/business/fed-cut...
384,48,2025-10-29-06-52-00 +0000,wsj,Trump Expects to Sign China Trade Deal at Xi S...,https://www.wsj.com/economy/trade/trump-expect...
266,45,2025-10-29-15-08-02 +0000,nypost,Nvidia becomes first $5T company powered by AI...,https://nypost.com/2025/10/29/business/nvidia-...
251,44,2025-10-29-16-04-58 +0000,bbc,"Israeli strikes in Gaza kill 104, health minis...",https://www.bbc.com/news/articles/cgjdy5eevn2o...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
